In [2]:
import pandas as pd
import os
import numpy as np
from collections import OrderedDict
from src import project_dir
from src.selection import kept_features, all_features
feature_area_map = {
    'min': 0.002,
    'max': 0.002,
    'sum': 0.0099,
    'mean': 0.0084
}
features_power_map = {
    'min': 0.44,
    'max': 0.44,
    'sum': 232,
    'mean': 155
}

2025-08-13 09:15:32.221713: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755065732.243234 1562679 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755065732.249881 1562679 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1755065732.267401 1562679 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755065732.267421 1562679 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755065732.267424 1562679 computation_placer.cc:177] computation placer alr

In [7]:
dataset_file = os.path.join(project_dir, 'data', 'wesad_merged.csv')
use_all_features = False
selected_features = [35, 7, 38, 32, 11, 4, 34, 36, 6, 8, 39, 10, 44, 52, 46, 47, 54, 55, 16, 33]


sensors = pd.read_csv(dataset_file, nrows=1).columns.to_list()
num_sensors = len(sensors) - 2
print(f"{num_sensors} sensors found in the dataset: {sensors}")
features_dict = OrderedDict([
    (sensor_id, all_features) if use_all_features else
    (sensor_id, kept_features)
    for sensor_id in range(0, num_sensors)
])

all_feature_names = [
    feature
    for sensor_id, features in features_dict.items()
    for feature in features
]

selected_feature_list = np.array(all_feature_names)[selected_features]
print(f"{len(selected_feature_list)} Selected features:\n{selected_feature_list.tolist()}")

if use_all_features:
    print("Cannot compute cost of selected features with all features enabled.")

else:
    total_area = sum(feature_area_map[feature] for feature in selected_feature_list)
    print(f"Total area of selected features: {total_area}")
    total_power = sum(features_power_map[feature] for feature in selected_feature_list)
    print(f"Total power of selected features: {total_power}")

14 sensors found in the dataset: ['ax', 'ay', 'az', 'emg', 'temp', 'eda', 'ecg', 'resp', 'ax_wrist', 'ay_wrist', 'az_wrist', 'eda_wrist', 'temp_wrist', 'bvp_wrist', 'label', 'subject']
20 Selected features:
['mean', 'mean', 'sum', 'min', 'mean', 'min', 'sum', 'min', 'sum', 'min', 'mean', 'sum', 'min', 'min', 'sum', 'mean', 'sum', 'mean', 'min', 'max']
Total area of selected features: 0.12580000000000002
Total power of selected features: 2325.5200000000004


### Compute digital features area/power/delay

In [51]:
features_csv = '/home/balaskas/pestress/hw_eval/features/results.csv'
sampling_rate = 32
precision = 4
features_list = ['muPeaks', 'mean', 'npeaks', 'npeaks', 'std', 'npeaks', 'peaks', 'peaks', 'min', 'std', 'mean', 'std', 'min', 'peaks', 'mean', 'npeaks', 'min', 'mean', 'muPeaks', 'peaks']

df = pd.read_csv(features_csv)
prec_df = df[(df['Precision'] == precision)]
single_sample_features = [feature for feature in df['FeatureName'].unique()
                          if set(df[df['FeatureName'] == feature]['NumSamples']) == {1}]

total_area = total_power = total_delay = total_energy = 0
for _feature in features_list:
    if _feature == 'median':
        print("Skipping 'median' feature")
        continue
    if _feature == 'peaks':
        feature = "npeaks"
    else:
        feature = _feature

    num_samples = 1 if feature in single_sample_features else sampling_rate
    feature_data = prec_df[(prec_df['FeatureName'] == feature) & (prec_df['NumSamples'] == num_samples)]
    if feature_data.empty:
        print(f"Feature {feature} not found in the dataset.")
        continue
    total_area += feature_data['Area(um2)'].iloc[0]
    total_power += feature_data['Power(mW)'].iloc[0]
    total_delay = max(total_delay, feature_data['Delay(ns)'].iloc[0])

    total_energy = feature_data['Power(mW)'].iloc[0] * feature_data['Delay(ns)'].iloc[0] * sampling_rate

print(f"Total area of selected features: {total_area:.3e} um^2")
print(f"Total power of selected features: {total_power:.3e} mW")
print(f"Total delay of selected features: {total_delay:.3e} ns")
print(f"Total energy of selected features: {total_energy * 1E-6:.3e} uJ")  # mW * ns = 1E-12 J = 1E-6 uJ

Total area of selected features: 7.243e+06 um^2
Total power of selected features: 2.253e-04 mW
Total delay of selected features: 7.288e+03 ns
Total energy of selected features: 5.210e-07 uJ


In [50]:
for feature in ['min', 'max', 'sum', 'mean']:
    num_samples = 1 if feature in single_sample_features else sampling_rate
    feature_df = df[(df['Precision'] == precision) & (df['FeatureName'] == feature) & (df['NumSamples'] == num_samples)]
    print(feature_df[['FeatureName', 'Area(um2)', 'Power(mW)',]])

    FeatureName     Area(um2)  Power(mW)
203         min  90500.960022   0.000002
   FeatureName    Area(um2)  Power(mW)
67         max  88998.56012   0.000002
    FeatureName      Area(um2)  Power(mW)
395         sum  238207.980103   0.000007
    FeatureName      Area(um2)  Power(mW)
115        mean  238207.980103   0.000007
